# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
# Libraries for loading
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# Libraries for tokenization function
import re
import nltk
nltk.download(['wordnet', 'stopwords', 'punkt'])
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

# Libraries for ml pipeline
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
import pickle

# Random State
np.random.seed(42)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('SELECT * FROM YourTableName', engine)

# Since it takes forever to train and gridsearch all dataset
# i've decided to use 10% the data until the final training.
df = df.sample(frac=0.1)

# message is our dependent variable so X contains it
# categories are our independent variables so Y contains them
# i've decided not to use genre variable in pipeline
X = df['message']
Y = df.drop(['message', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    '''
    This function takes a text 
    cleans from punctiation, tokenize it
    and remove english stop words
    then returns list constructed by tokenized text elements.
    
    input : a collection of test messages such as list/dataframe.
    output : a cleaned and tokenized list.
    '''
    
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    words = word_tokenize(text)
    wnlm = WordNetLemmatizer()
    words = [wnlm.lemmatize(word) for word in words if word not in stopwords.words('english')]
    
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
# Creating pipeline instance with vectorizer, transformer and classifier.
# i've choose CountVectorizer and TfidfTransformer duo for vectorization like in lessons.
# i've choose RandomForestClassifier for first classfier. 
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize, max_features=5000)),
    ('transformer', TfidfTransformer()),
    ('classifer', MultiOutputClassifier(RandomForestClassifier(n_estimators=100)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# Spliting data into train and test set with 25% test size.
X_train, X_test, y_train, y_test = train_test_split(X.values, Y, test_size=0.25, shuffle=True)

# Fitting pipeline with train set.
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [6]:
print('Pipeline Train Score: ', pipeline.score(X_train, y_train))
print('Pipeline Test Score: ', pipeline.score(X_test, y_test))

Pipeline Train Score:  0.998981151299
Pipeline Test Score:  0.235114503817


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# Making prediction from our test data (25% of our original data)
y_pred = pipeline.predict(X_test)

# calculating and priting a classfication report instance for every column
y_pred = pd.DataFrame(y_pred, index=y_test.index, columns=y_test.columns)

for column in y_test.columns:
    cr_default = classification_report(y_test[column].values, y_pred[column].values)
    print('Column Name: ', column)
    print('Classification Report: ', cr_default)

Column Name:  related
Classification Report:               precision    recall  f1-score   support

          0       0.65      0.36      0.47       162
          1       0.81      0.93      0.87       491
          2       0.00      0.00      0.00         2

avg / total       0.77      0.79      0.77       655

Column Name:  request
Classification Report:               precision    recall  f1-score   support

          0       0.86      0.98      0.91       533
          1       0.73      0.29      0.41       122

avg / total       0.83      0.85      0.82       655

Column Name:  offer
Classification Report:               precision    recall  f1-score   support

          0       0.99      1.00      1.00       651
          1       0.00      0.00      0.00         4

avg / total       0.99      0.99      0.99       655

Column Name:  aid_related
Classification Report:               precision    recall  f1-score   support

          0       0.73      0.83      0.78       379
         

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
# to gridsearch i've decided to only try 3 simple parameters 
parameters = {
    'transformer__use_idf': (True, False)
}

cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, y_train)

print('Best Params are: ', cv.best_params_)
print('Best Score is: ', cv.best_score_)

Best Params are:  {'transformer__use_idf': True}
Best Score is:  0.2109016811


In [9]:
print('GridSearch Train Score: ', cv.score(X_train, y_train))
print('GridSearch Test Score: ', cv.score(X_test, y_test))

GridSearch Train Score:  0.99949057565
GridSearch Test Score:  0.229007633588


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
# Gridsearch prediction from our test data (25% of our original data)
y_pred_cv = cv.predict(X_test)

# calculating and priting a classfication report instance for every column
y_pred_cv = pd.DataFrame(y_pred_cv, index=y_test.index, columns=y_test.columns)

for column in y_test.columns:
    cr_1 = classification_report(y_test[column].values, y_pred_cv[column].values)
    print('Column Name: ', column)
    print('GridSearch Classification Report: ', cr_1)

Column Name:  related
GridSearch Classification Report:               precision    recall  f1-score   support

          0       0.69      0.37      0.48       162
          1       0.82      0.95      0.88       491
          2       0.00      0.00      0.00         2

avg / total       0.78      0.80      0.78       655

Column Name:  request
GridSearch Classification Report:               precision    recall  f1-score   support

          0       0.86      0.98      0.91       533
          1       0.77      0.30      0.43       122

avg / total       0.84      0.85      0.82       655

Column Name:  offer
GridSearch Classification Report:               precision    recall  f1-score   support

          0       0.99      1.00      1.00       651
          1       0.00      0.00      0.00         4

avg / total       0.99      0.99      0.99       655

Column Name:  aid_related
GridSearch Classification Report:               precision    recall  f1-score   support

          0       

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Since our gridsearch instance has chosen same metric on max features, our pipeline and gridsearch instance has the same results.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
# lets try to apply 1 new ml classifier and see if we can beat RF Classifier
# Same logic and code flow with first pipeline
parameters_2 = {
    'classifer__estimator': [RandomForestClassifier(), DecisionTreeClassifier()]
}

cv_2 = GridSearchCV(pipeline, param_grid=parameters_2)

cv_2.fit(X_train, y_train)

print('Best Params are: ', cv_2.best_params_)
print('Best Score is: ', cv_2.best_score_)

Best Params are:  {'classifer__estimator': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)}
Best Score is:  0.196637799287


In [12]:
print('GridSearch 2 Train Score: ', cv_2.score(X_train, y_train))
print('GridSearch 2 Test Score: ', cv_2.score(X_test, y_test))

GridSearch 2 Train Score:  0.779928680591
GridSearch 2 Test Score:  0.238167938931


In [13]:
# Predictions with best estimator
y_pred_cv_2 = cv_2.predict(X_test)

y_pred_cv_2 = pd.DataFrame(y_pred_cv_2, index=y_test.index, columns=y_test.columns)

for column in y_test.columns:
    cr_2 = classification_report(y_test[column].values, y_pred_cv_2[column].values)
    print('Column Name: ', column)
    print('GridSearch Classification Report: ', cr_2)

Column Name:  related
GridSearch Classification Report:               precision    recall  f1-score   support

          0       0.61      0.43      0.51       162
          1       0.83      0.91      0.87       491
          2       0.00      0.00      0.00         2

avg / total       0.77      0.79      0.77       655

Column Name:  request
GridSearch Classification Report:               precision    recall  f1-score   support

          0       0.85      0.98      0.91       533
          1       0.71      0.26      0.38       122

avg / total       0.83      0.84      0.81       655

Column Name:  offer
GridSearch Classification Report:               precision    recall  f1-score   support

          0       0.99      1.00      1.00       651
          1       0.00      0.00      0.00         4

avg / total       0.99      0.99      0.99       655

Column Name:  aid_related
GridSearch Classification Report:               precision    recall  f1-score   support

          0       

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [14]:
# it seems grid search with use_idf True and estimator RF is the best option
# so i've decided to train final model with it
final_pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize, max_features=5000)),
    ('transformer', TfidfTransformer(use_idf=True)),
    ('classifer', MultiOutputClassifier(RandomForestClassifier(n_estimators=100)))
])

# Reloading database
df = pd.read_sql('SELECT * FROM YourTableName', engine)

# Selecting all data points
X = df['message']
y = df.drop(['message', 'genre'], axis=1)

# Spliting data into train and test set with 25% test size.
X_train, X_test, y_train, y_test = train_test_split(X.values, y, test_size=0.25, shuffle=True)

# Fitting pipeline with train set.
final_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [15]:
print('FinalModel Train Score: ', final_pipeline.score(X_train, y_train))
print('FinalModel Test Score: ', final_pipeline.score(X_test, y_test))

FinalModel Train Score:  0.991340668297
FinalModel Test Score:  0.280213903743


In [16]:
# Saving final model.
f = 'your_model_name.pkl'
with open(f, 'wb') as file:
    pickle.dump(final_pipeline, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.